In [1]:
import pandas as pd
from datetime import datetime
import dateutil

In [2]:
dataset = pd.read_csv("Task-dataset.csv")
dataset["timestamp"] = dataset["timestamp"].map(lambda x: dateutil.parser.parse(x, dayfirst=False))
dataset = dataset.sort_values(by=["timestamp"])
dataset.head()

,timestamp,user_id,event,flyer_id,merchant_id
13048,2018-10-01 00:01:06-04:00,0307f16bb87891bef7a537ebb04b24ba9ed4514cf4e7ef...,flyer_open,1995315,2609
13054,2018-10-01 00:01:06-04:00,26f10bb26320cb3e9338a9d1a9eb74339300e5e149233c...,flyer_open,1990045,249
13049,2018-10-01 00:01:16-04:00,0307f16bb87891bef7a537ebb04b24ba9ed4514cf4e7ef...,flyer_open,1960380,2484
11796,2018-10-01 00:02:08-04:00,b38f97880ccb69e8f89201eedbccc149a9ea914ca86111...,item_open,1960380,2484
11781,2018-10-01 00:03:43-04:00,b38f97880ccb69e8f89201eedbccc149a9ea914ca86111...,flyer_open,2005556,3048


## Choosing the right assumption

- only the "flyer_open" event closes the previous flyer.
- every new event closes the pervious flyer.

In [3]:
EVERY_EVENT_CLOSES_PREVIOUS_FLYER = True

## Counting time users spent on flyers


Create new df with only flyer_open

In [4]:
if not EVERY_EVENT_CLOSES_PREVIOUS_FLYER:
    dataset = dataset[dataset["event"] == "flyer_open"]
dataset.head()

,timestamp,user_id,event,flyer_id,merchant_id
13048,2018-10-01 00:01:06-04:00,0307f16bb87891bef7a537ebb04b24ba9ed4514cf4e7ef...,flyer_open,1995315,2609
13054,2018-10-01 00:01:06-04:00,26f10bb26320cb3e9338a9d1a9eb74339300e5e149233c...,flyer_open,1990045,249
13049,2018-10-01 00:01:16-04:00,0307f16bb87891bef7a537ebb04b24ba9ed4514cf4e7ef...,flyer_open,1960380,2484
11796,2018-10-01 00:02:08-04:00,b38f97880ccb69e8f89201eedbccc149a9ea914ca86111...,item_open,1960380,2484
11781,2018-10-01 00:03:43-04:00,b38f97880ccb69e8f89201eedbccc149a9ea914ca86111...,flyer_open,2005556,3048


In [5]:
def count_time_spent_when_open_flyer_closes(group_entries):
    result = []
    
    previous_opened_flyer = None
    previous_opened_time = None
    previous_opened_merchant = None
    
    for entry in group_entries:
        if previous_opened_flyer is not None: #user never opend a flyer
            time_spent = int((entry[0] - previous_opened_time).seconds) #returns float =
            result.append([time_spent, previous_opened_time, entry[0], entry[1], previous_opened_flyer, previous_opened_merchant])
            
        previous_opened_flyer = entry[3]
        previous_opened_time = entry[0]
        previous_opened_merchant = entry[4]

    return result  

In [6]:
def count_time_spent_when_every_event_closes(group_entries):
    result = []
    
    previous_opened_flyer = None
    previous_opened_time = None
    previous_opened_merchant = None
    
    for entry in group_entries:
        if entry[2] == "flyer_open": # counter starts when flyer_open event happens & same code as above is triggered
            if previous_opened_flyer is not None:
                time_spent = int((entry[0] - previous_opened_time).seconds)
                result.append([time_spent, previous_opened_time, entry[0], entry[1], previous_opened_flyer, previous_opened_merchant])

            previous_opened_flyer = entry[3]
            previous_opened_time = entry[0]
            previous_opened_merchant = entry[4]
        else:
            if previous_opened_flyer is not None:
                time_spent = int((entry[0] - previous_opened_time).seconds)
                result.append([time_spent, previous_opened_time, entry[0], entry[1], previous_opened_flyer, previous_opened_merchant])
                previous_opened_flyer = None # any event closes the flyer

    return result  

In [7]:
new_values = [] # list
for name, group in dataset.groupby("user_id"):
    if EVERY_EVENT_CLOSES_PREVIOUS_FLYER:
        new_values += count_time_spent_when_every_event_closes(group.values)
    else:
        new_values += count_time_spent_when_open_flyer_closes(group.values)
        
# new df created from the grouped list that was a result of the function
new_dataframe = pd.DataFrame(new_values, columns=["seconds_spent", "from_timestamp", "end_timestamp", "user_id", "flyer_id", "merchant_id"]) 
new_dataframe.head()

,seconds_spent,from_timestamp,end_timestamp,user_id,flyer_id,merchant_id
0,41,2018-10-01 11:34:20-04:00,2018-10-01 11:35:01-04:00,0017345b89958a1d8cae79020dbbf6e2f687124ae8bf93...,2000818,2428
1,1,2018-10-01 11:36:04-04:00,2018-10-01 11:36:05-04:00,0017345b89958a1d8cae79020dbbf6e2f687124ae8bf93...,1994646,2608
2,0,2018-10-01 11:36:05-04:00,2018-10-01 11:36:05-04:00,0017345b89958a1d8cae79020dbbf6e2f687124ae8bf93...,2008626,2353
3,19,2018-10-01 11:36:06-04:00,2018-10-01 11:36:25-04:00,0017345b89958a1d8cae79020dbbf6e2f687124ae8bf93...,2016311,2268
4,1,2018-10-01 11:36:25-04:00,2018-10-01 11:36:26-04:00,0017345b89958a1d8cae79020dbbf6e2f687124ae8bf93...,1994508,986


## Task 1

Compute the unique users in the dataset.


In [8]:
unique_users_start = dataset["user_id"].nunique()
unique_users_processed = new_dataframe["user_id"].nunique()

print(f"Found {unique_users_start} unique users in the dataframe")
print(f"Found {unique_users_processed} unique users in the dataframe who had at least 2 entries") # at least 1 flyer_open opened and closed it

Found 1190 unique users in the dataframe
Found 1057 unique users in the dataframe who had at least 2 entries


## Task 2

Compute the average time on flyer per user.

In [9]:
avg_time = new_dataframe.groupby("flyer_id").apply(lambda x: sum(x["seconds_spent"] / len(x))).to_dict() # x is a row ; len(x)
print(f"Counted {len(avg_time)} unique flyers")
print(f"Average time per user spent on a flyer is {sum(avg_time.values()) / len(avg_time):.2f} seconds")

Counted 1637 unique flyers
Average time per user spent on a flyer is 1847.08 seconds


2nd interpretation of Task 2

In [10]:
avg_time_by_user = new_dataframe.groupby("user_id").apply(function).reset_index()
avg_time_by_user.columns = ["user_id", "average_time_spent"]
avg_time_by_user.head()

In [11]:
avg_time_by_user = new_dataframe.groupby("user_id").apply(function).reset_index()
avg_time_by_user.columns = ["user_id", "average_time_spent"]
avg_time_by_user.head()

,user_id,average_time_spent
0,0017345b89958a1d8cae79020dbbf6e2f687124ae8bf93...,64.15
1,002c17c9db976a1cad3516033b5023c5e11eff08fdbcb1...,2809.50
2,00a52eb26d5b84881fea69429e91d2e7e7cb1004db83f8...,554.00
3,00d6164ad0d97a07a8ff2fbc6e10352736e4a8c02be283...,63.00
4,00dbcd0f49522e8614a0e363293b2a371f4083919b5d27...,43408.00


## Task 3

Compute the average time users spent on flyer_id = 2020004

In [12]:
flyer_to_check = "2020004"
print(f"Users spent {avg_time.get(flyer_to_check, 0):.2f} seconds on average viewing flyer with id = {flyer_to_check}")

Users spent 1568.06 seconds on average viewing flyer with id = 2020004


## Task 4

Explain how your algorithm scales for:

a) 1 Million Events (~10 MB of data)

b) 1 Trillion Events (~10 TB of data) 

**Answer**

Algorithm needs:

- Sorting timestamps ~O(N logN)
- grouping users ~O(N LogN)
- iteration on every entry ~O(N)

Algorithm scalability is almost linear. difficulty is ~O(N LogN)

If datasets get very big we could improve the script by parallelizing some of the tasks.